In [2]:
import pandas as pd 
import numpy as np
import re
from geopy.distance import geodesic
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

df = pd.read_csv(r"D:\delivery\train.csv")
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [3]:
df.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [4]:
df['Time_taken(min)'] = df['Time_taken(min)'].str.replace('(min)','')
df['Weatherconditions']=df['Weatherconditions'].str.replace('conditions','')
df.rename(columns={'Time_taken(min)':'Time_taken', 'Weatherconditions': 'Weather_conditions'}, inplace=True)


In [11]:
df.replace({'NaN': np.nan}, regex=True, inplace = True)
df.isnull().sum().sort_values(ascending=False)

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken                     0
dtype: int64

In [6]:
lookup_table = df[["Delivery_person_ID", "Delivery_person_Age"]].dropna().drop_duplicates(subset = "Delivery_person_ID")

# Merge the original dataframe "food" with the lookup table
df = df.merge(lookup_table, on = "Delivery_person_ID", how = "left", suffixes=("", "_filled"))

# Fill missing values in "delivery_person_age" with the ones from the merged table
df["Delivery_person_Age"] = df["Delivery_person_Age"].combine_first(df["Delivery_person_Age_filled"])

# Dropping the temporary "delivery_person_age_filled" column
df.drop(columns = ["Delivery_person_Age_filled"], inplace = True)
df["Delivery_person_Age"] = df["Delivery_person_Age"].astype(int)

In [7]:
df["Delivery_person_Ratings"] = pd.to_numeric(df["Delivery_person_Ratings"], errors = "coerce")
# df['Delivery_person_Ratings'] = pd.to_numeric(df['Delivery_person_Ratings'], errors = 'coerce')
lookup_table = df.groupby('Delivery_person_ID', as_index = False)['Delivery_person_Ratings'].mean()

df = df.merge(lookup_table, on = 'Delivery_person_ID', how = "left", suffixes=("", "_filled"))

df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].combine_first(df['Delivery_person_Ratings_filled'])

df.drop(columns=['Delivery_person_Ratings_filled'], inplace=True)

In [8]:
lookup_table = df[["Order_Date","Weather_conditions"]].dropna().drop_duplicates(subset = "Order_Date")

# Merge the original dataframe "food" with look up table
df = df.merge(lookup_table, on = "Order_Date", how = "left", suffixes=("", "_filled"))

# Fill missing values in "weather_conditions" with the ones from the merged table
df["Weather_conditions"] = df["Weather_conditions"].combine_first(df["Weather_conditions_filled"])

# Dropping the temporary "weather_conditions_filled" column
df.drop(columns = ["Weather_conditions_filled"], inplace = True)

In [9]:
df['Time_Orderd'] = df['Time_Orderd'].fillna(df['Time_Orderd'].mode()[0])
df['Road_traffic_density'] = df['Road_traffic_density'].fillna(df['Road_traffic_density'].mode()[0])
df['multiple_deliveries'] = df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0])
df['City'] = df['City'].fillna(df['City'].mode()[0])
df['Festival'] = df['Festival'].fillna(df['Festival'].mode()[0])

In [10]:

df['Time_taken'] = df['Time_taken'].astype(int)
df['Order_Date']= pd.to_datetime(df['Order_Date'], format="%d-%m-%Y")
df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)
# df['Time_Order_picked'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M:%S')
# df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'], format='%H:%M:%S')

In [12]:
df['City_code']=df['Delivery_person_ID'].str.split("RES", expand=True)[0]
df['City_code'].head()

0     INDO
1     BANG
2     BANG
3    COIMB
4     CHEN
Name: City_code, dtype: object

In [14]:
if (len(df[df.duplicated()])>0):
    print("There are Duplicate values present")
else:
    print("There is no duplicate value")

There is no duplicate value


In [15]:
def extract_date_features(df):
    df["day"] = df.Order_Date.dt.day
    df["month"] = df.Order_Date.dt.month
    df["quarter"] = df.Order_Date.dt.quarter
    df["year"] = df.Order_Date.dt.year
    df['day_of_week'] = df.Order_Date.dt.day_of_week.astype(int)
    df["is_month_start"] = df.Order_Date.dt.is_month_start.astype(int)
    df["is_month_end"] = df.Order_Date.dt.is_month_end.astype(int)
    df["is_quarter_start"] = df.Order_Date.dt.is_quarter_start.astype(int)
    df["is_quarter_end"] = df.Order_Date.dt.is_quarter_end.astype(int)
    df["is_year_start"] = df.Order_Date.dt.is_year_start.astype(int)
    df["is_year_end"] = df.Order_Date.dt.is_year_end.astype(int)
    df['is_weekend'] = np.where(df['day_of_week'].isin([5,6]),1,0)

extract_date_features(df)

In [16]:
df['Time_Orderd']=pd.to_timedelta(df['Time_Orderd'])
df['Time_Order_picked']=pd.to_timedelta(df['Time_Order_picked'])

df['Time_Order_picked_formatted']=np.where(df['Time_Order_picked'] < df['Time_Orderd'], df['Order_Date'] + pd.DateOffset(1)+df['Time_Order_picked'], df['Order_Date']+df['Time_Order_picked'])
df['Time_Ordered_formatted'] = df['Order_Date']+ df['Time_Orderd']
df['order_preparation_time']=((df['Time_Order_picked_formatted']- df['Time_Ordered_formatted']).dt.total_seconds())/60
    # Handle null values by filling with the median
    # df['order_prepare_time'].fillna(df['order_prepare_time'].median(), inplace=True)
    
    # Drop all the time & date related columns
df.drop(['Time_Orderd', 'Time_Order_picked', 'Time_Ordered_formatted', 'Time_Order_picked_formatted', 'Order_Date'], axis=1, inplace=True)



In [17]:
df['order_preparation_time'].head()

0    15.0
1     5.0
2    15.0
3    10.0
4    15.0
Name: order_preparation_time, dtype: float64

In [18]:
def calculate_distance(df):
    df['distance']=np.zeros(len(df))
    restaurant_coordinates=df[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
    delivery_location_coordinates=df[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()
    df['distance'] = np.array([geodesic(restaurant, delivery) for restaurant, delivery in zip(restaurant_coordinates, delivery_location_coordinates)])
    df['distance']= df['distance'].astype("str").str.extract(r'(\d+)').astype("int64")
    
calculate_distance(df)
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,...,day_of_week,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_weekend,order_preparation_time,distance
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,...,5,0,0,0,0,0,0,1,15.0,3
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,...,4,0,0,0,0,0,0,0,5.0,20
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,...,5,0,0,0,0,0,0,1,15.0,1
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,...,1,0,0,0,0,0,0,0,10.0,7
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,...,5,0,0,0,0,0,0,1,15.0,6


In [19]:
def label_encoding(df):
    categorical_columns = df.select_dtypes(include='object').columns
    label_encoder = LabelEncoder()
    df[categorical_columns] = df[categorical_columns].apply(lambda col: label_encoder.fit_transform(col))

label_encoding(df)

In [20]:
X = df.drop('Time_taken', axis=1)  # Features
y = df['Time_taken']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
    )

In [21]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Perform standardization on the training data
X_train = scaler.transform(X_train)

# Perform standardization on the testing data
X_test = scaler.transform(X_test)

In [28]:
from sklearn.model_selection import GridSearchCV

models = [
    DecisionTreeRegressor(),
    RandomForestRegressor(),
]


param_grid = [
    {'max_depth': [3, 5, 7]},  # DecisionTree 
    {'n_estimators': [100, 200, 300]},  # RandomForest 
]
best_models = {} 
# GridSearch ishlatish
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, param_grid[i], cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)

    print(f"{model.__class__.__name__}:")
    print("Best parameters:", grid_search.best_params_)
    print("Best R2 score:", grid_search.best_score_)
    print()
    best_models[model.__class__.__name__] = grid_search.best_estimator_


DecisionTreeRegressor:
Best parameters: {'max_depth': 7}
Best R2 score: 0.7206412489777273

RandomForestRegressor:
Best parameters: {'n_estimators': 300}
Best R2 score: 0.8161490129196443



In [32]:
# DecisionTree 
dt_best = best_models["DecisionTreeRegressor"]
y_pred_dt = dt_best.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_dt)
mse = mean_squared_error(y_test, y_pred_dt)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_dt)

print("📌 DecisionTree results:")
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("RMSE:", round(rmse, 2))
print("R2:", round(r2, 2))

# print("\n----------------------\n")

# RandomForest 
rf_best = best_models["RandomForestRegressor"]
y_pred_rf = rf_best.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_rf)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_rf)

print("📌 RandomForest results:")
print("MAE:", round(mae, 2))
print("MSE:", round(mse, 2))
print("RMSE:", round(rmse, 2))
print("R2:", round(r2, 2))


📌 DecisionTree results:
MAE: 3.91
MSE: 24.79
RMSE: 4.98
R2: 0.72
📌 RandomForest results:
MAE: 3.2
MSE: 16.4
RMSE: 4.05
R2: 0.81
